In [1]:
from argparse import ArgumentParser
from model.origgnn import MolecularGNN
from utils.package import plot_fit_confidence_bond
from pytorch_lightning import Trainer
import pytorch_lightning as pl
from collections import defaultdict
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch
import numpy as np
import wandb
import time
from sklearn.metrics import r2_score
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')

In [2]:

wandb.login(key='local-8fe6e6b5840c4c05aaaf6aac5ca8c1fb58abbd1f', host='http://localhost:8080')


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: huabei. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for localhost to your netrc file: /home/huabei/.netrc


True

In [3]:
# %%wandb
def main(hparams):
    
    model_name = f'3dgnn-dim-{hparams.dim}-hlayer-{hparams.layer_hidden}-olayer-{hparams.layer_output}-' + time.strftime("%Y%m%d_%H%M%S", time.localtime())
    dict_args = vars(hparams)
    model = MolecularGNN(**dict_args)
    # logger
    wandb_logger = pl.loggers.WandbLogger(save_dir='log/origgnn')
    # callbacks
    # early stopping
    early_stopping = pl.callbacks.early_stopping.EarlyStopping(monitor='val_loss', patience=20, mode='min')
    # checkpoint
    checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor='val_loss', mode='min', save_last=True,
                                                         dirpath='checkpoints', filename=model_name)
    if hparams.checkpoint == None:
        trainer = Trainer.from_argparse_args(hparams, logger=wandb_logger, auto_lr_find=True, callbacks=[early_stopping, checkpoint_callback])
    else:
        trainer = Trainer(resume_from_checkpoint=hparams.checkpoint, callbacks=[early_stopping])
    # trainer.tune(model)
 
    # Train
    trainer.fit(model)
    # trainer.save_checkpoint(time.strftime("%Y%m%d_%H%M%S", time.localtime()) + ".ckpt")
    trainer.test(model, dataloaders=model.val_dataloader(), verbose=False)
    x = np.array(model.predictions['true'])
    y = np.array(model.predictions['pred'])
    val_r2 = r2_score(x, y)
    val_fig = plot_fit_confidence_bond(x, y, val_r2, annot=False)
    
    model.predictions = defaultdict(list)
    trainer.test(model, dataloaders=model.train_dataloader(), verbose=False)
    x = np.array(model.predictions['true'])
    y = np.array(model.predictions['pred'])
    train_r2 = r2_score(x, y)
    train_fig = plot_fit_confidence_bond(x, y, train_r2, annot=False)
    if True:
        wandb.log({'train_res': train_fig, 'val_res': val_fig})
        wandb.log({'val_r2': val_r2, 'train_r2':train_r2})
        wandb.finish()

def prepare_arg():
    parser = ArgumentParser()
    parser.add_argument("--dim", type=int, default=512)
    parser.add_argument("--layer_hidden",type=int, default=24)
    parser.add_argument("--layer_output",type=int, default=8)
    parser.add_argument("--batch_size",type=int, default=128)
    parser.add_argument("--data_path",type=str, default=None)
    parser.add_argument("--checkpoint",type=str, default=None)
    return parser

In [ ]:

if __name__ == "__main__":
    project = '3dgnn'
    wandb.init(project=project, dir='log/origgnn')
    # prepare args
    parser = prepare_arg()
    dataset_path = '/home/huabei/Projects/SMTarRNA/project/data/in_man_exhaustiveness_96_orig_conformation.txt'
    checkpoint = '20220618_211314.ckpt'
    # add model args
    parser = MolecularGNN.add_model_specific_args(parent_parser=parser)
    # add Trainer args
    parser = Trainer.add_argparse_args(parser)
    args = parser.parse_args(['--data_path', dataset_path, '--learning_rate', '0.0001', '--gpus=1', '--max_epochs', '1000'])

    main(args)

# wandb sweep

In [4]:
sweep_config = {
  "name" : "sweep",
  "method" : "random",
  "parameters": {
    "epochs": {
      "value": 500
    },
    "learning_rate": {
      "distribution": "log_uniform_values",
      "min": 0.00001,
      "max": 0.1
    },
    "lr_decay": {
      "min": 0.95,
      "max": 0.999
    },
    "dim" : {
      "distribution": "int_uniform",
      "min": 128,
      "max": 512
    },
    "layer_hidden": {
      "distribution": "int_uniform",
      "min": 8,
      "max": 32
    },
    "layer_output": {
      "distribution": "int_uniform",
      "min": 8,
      "max": 20
    }
  }
}

sweep_id = wandb.sweep(sweep_config, project='origgnn_sweep')

Create sweep with ID: t9aqqtww
Sweep URL: http://localhost:8080/huabei/origgnn_sweep/sweeps/t9aqqtww


In [11]:
def arg_for_sweep(config: dict):
    # prepare args
    parser = prepare_arg()
    dataset_path = '/home/huabei/Projects/SMTarRNA/project/data/in_man_exhaustiveness_96_orig_conformation.txt'
    checkpoint = '20220618_211314.ckpt'
    # add model args
    parser = MolecularGNN.add_model_specific_args(parent_parser=parser)
    # add Trainer args
    parser = Trainer.add_argparse_args(parser)
    hyperparameter_list = ['--data_path', dataset_path, '--gpus=1']
    for key, value in config.items():
        # print(key, value)
        hyperparameter_list.extend(['--' + key, str(value)])
    # print(hyperparameter_list)
    args = parser.parse_args(hyperparameter_list)
    # print('here is right')
    return args
    # args = parser.parse_args(['--data_path', dataset_path, '--learning_rate', '0.0001', '--gpus=1', '--max_epochs', '1000'])

def train():
    with wandb.init(dir='log/origgnn') as run:
        config = wandb.config
        args = arg_for_sweep(config=config)
        # print(vars(args))
        # print(config)
        # main(args)

In [12]:
count = 1 # number of runs to execute
wandb.agent(sweep_id, function=train, count=count)

wandb: Agent Starting Run: 0w9lznno with config:
wandb: 	dim: 484
wandb: 	epochs: 500
wandb: 	layer_hidden: 23
wandb: 	layer_output: 13
wandb: 	learning_rate: 0.0010658098334630822
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


dim 484
epochs 500
layer_hidden 23
layer_output 13
learning_rate 0.0010658098334630822
['--data_path', '/home/huabei/Projects/SMTarRNA/project/data/in_man_exhaustiveness_96_orig_conformation.txt', '--gpus=1', '--dim', 484, '--epochs', 500, '--layer_hidden', 23, '--layer_output', 13, '--learning_rate', 0.0010658098334630822]


Run 0w9lznno errored: TypeError("'int' object is not subscriptable")
wandb: ERROR Run 0w9lznno errored: TypeError("'int' object is not subscriptable")
wandb: Agent Starting Run: bqpdn6sk with config:
wandb: 	dim: 357
wandb: 	epochs: 500
wandb: 	layer_hidden: 26
wandb: 	layer_output: 11
wandb: 	learning_rate: 0.0005298522584671892
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


dim 357
epochs 500
layer_hidden 26
layer_output 11
learning_rate 0.0005298522584671892
['--data_path', '/home/huabei/Projects/SMTarRNA/project/data/in_man_exhaustiveness_96_orig_conformation.txt', '--gpus=1', '--dim', 357, '--epochs', 500, '--layer_hidden', 26, '--layer_output', 11, '--learning_rate', 0.0005298522584671892]


Run bqpdn6sk errored: TypeError("'int' object is not subscriptable")
wandb: ERROR Run bqpdn6sk errored: TypeError("'int' object is not subscriptable")
wandb: Agent Starting Run: dru02rvo with config:
wandb: 	dim: 461
wandb: 	epochs: 500
wandb: 	layer_hidden: 29
wandb: 	layer_output: 13
wandb: 	learning_rate: 8.877196129090286e-05
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


dim 461
epochs 500
layer_hidden 29
layer_output 13
learning_rate 8.877196129090286e-05
['--data_path', '/home/huabei/Projects/SMTarRNA/project/data/in_man_exhaustiveness_96_orig_conformation.txt', '--gpus=1', '--dim', 461, '--epochs', 500, '--layer_hidden', 29, '--layer_output', 13, '--learning_rate', 8.877196129090286e-05]


Run dru02rvo errored: TypeError("'int' object is not subscriptable")
wandb: ERROR Run dru02rvo errored: TypeError("'int' object is not subscriptable")
Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: ERROR Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: To disable this check set WANDB_AGENT_DISABLE_FLAPPING=true
